In [ ]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.base import ModeDataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.data.spatial.modes import AvirisDataManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from spectraclass.learn.manager import ClassificationManager, cm
from spectraclass.learn.base import LearningModel
import os
from typing import List, Union, Tuple, Optional, Dict, Callable

Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [2]:
dm: DataManager = DataManager.initialize( "img_mgr", 'aviris' )
location = "desktop"
if location == "adapt":
    dm.modal.cache_dir = "/adapt/nobackup/projects/ilab/cache"
    dm.modal.data_dir = "/css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data/"
elif location == "desktop":
    dm.modal.cache_dir = "/Volumes/Shared/Cache"
    dm.modal.data_dir = "/Users/tpmaxwel/Development/Data/Aviris"
else: raise Exception( f"Unknown location: {location}")
    
dm.modal.ext =  "_img"
dm.proc_type = "cpu"
TileManager.block_size = 128
TileManager.block_index = [1,4]
AvirisDataManager.valid_aviris_bands = [ [5,193], [214,283], [319,10000] ]

Opening log file:  '/Users/tpmaxwel/.spectraclass/logging/aviris/img_mgr.log'
Using config file: '/Users/tpmaxwel/Development/Projects/spectraclass/defaults/config.py'
Using config file: '/Users/tpmaxwel/.spectraclass/config/aviris/img_mgr.py'


/Applications/anaconda3/envs/spectraclass/lib/python3.10/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Here we load the project data and define a set of class names (with associated colors) to be used in the classification process.

In [3]:
dm.loadCurrentProject()
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]

lm().setLabels( classes )

Loading metadata from file: /Volumes/Shared/Cache/spectraclass/aviris/img_mgr/ang20170720t004130rfl/ang20170720t004130_rfl_v2p9/ang20170720t004130_corr_v2p9_img.mdata.txt
Loading labels file: /Volumes/Shared/Cache/spectraclass/aviris/img_mgr/labels/20170720t004130_b-128-1-4-raw-labels.nc


Here we define a custom CNN to use withing the spectraclass framework.

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras import datasets, layers, models
from spectraclass.learn.models.network import Network, ModelType

class CNN(Network):
    TYPE = ModelType.SPATIAL

    def _build_model(self, **kwargs) -> Tuple[Model,Dict]:
        from spectraclass.learn.models.spatial import SpatialModelWrapper
        nfeatures = kwargs.pop('nfeatures', 32 )
        from spectraclass.model.labels import lm
        input_shape = SpatialModelWrapper.get_input_shape()
        nclasses = lm().nLabels
        ks = kwargs.pop('kernel_size',3)
        model = models.Sequential()
        model.add( Input( shape=input_shape ) )
        model.add( layers.Conv2D( nfeatures, (ks,ks), activation='relu', padding="same" ) )
        model.add( layers.Reshape( SpatialModelWrapper.flatten(input_shape,nfeatures) ) )
        model.add( layers.Dense( nfeatures, activation='relu' ) )
 #       model.add( layers.Dense( nfeatures, activation='relu') )
        model.add( layers.Dense( nclasses, activation='softmax' ) )
        return model, kwargs

cm().addNetwork( CNN('cnn-1', nfeatures=64 ) )

INFO:tensorflow:Assets written to: ram://f7758d2c-ac36-4642-8003-9df8738f0253/assets


Here we start up the Spectraclass GUI.

In [5]:
controller: SpectraclassController = app()
controller.gui()

Initializing GUI using controller <class 'spectraclass.gui.spatial.application.Spectraclass'>
Fri Jul 29 15:10:45 2022 Building RP forest with 11 trees
Fri Jul 29 15:10:45 2022 NN descent for 28 iterations
	 1  /  28
	 2  /  28
	 3  /  28
	 4  /  28
	Stopping threshold met -- exiting after 4 iterations
Fri Jul 29 15:11:09 2022 Building RP forest with 11 trees
Fri Jul 29 15:11:10 2022 NN descent for 28 iterations
	 1  /  28
	 2  /  28
	 3  /  28
	 4  /  28
	Stopping threshold met -- exiting after 4 iterations
Computing umap embedding with nepochs = 1, alpha = 1.0, nLabels = 0, n_neighbors = 5, init = random (random)
Completed UMAP random initialization, init shape = (15681, 3)
Completed simplicial_set_embedding, times = 0.00016710758209228515 4.327297210693359e-06 min
umap embedding complete, result shape = (15681, 3) 
